In [1]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
import sys
sys.path.append("../")
import d2l
device = torch.device('cuda' if torch.cuda.is_available() else
'cpu')

In [2]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

In [3]:
num_hiddens = 256
num_layers = 1
rnn_layer = nn.RNN(input_size=vocab_size,hidden_size=num_hiddens,num_layers=num_layers,bidirectional=False)

In [4]:
num_steps = 35
batch_size = 32
num_outputs = vocab_size
num_inputs = rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1)
rnn = d2l.RNNModel(rnn_layer, num_inputs, vocab_size).to(device)
# rnn.begin_state(num_hiddens, device, batch_size, num_layers=num_layers)

In [5]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

In [6]:
def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape),
                          device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens,
                                         device=device, requires_grad=True))
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs,
                                         device=device, requires_grad=True))
    return nn.ParameterList([W_xh, W_hh, b_h, W_hq, b_q])

In [ ]:
num_epochs, lr, clipping_theta = 250, 1e-3, 1e-2 
is_random_iter = False
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
d2l.train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          corpus_indices, char_to_idx, device, is_random_iter,
                          num_epochs, num_steps, lr, clipping_theta,
                          batch_size, prefixes)